In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"]  = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
import findspark
findspark.init()

In [0]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataCleaning_analysis').getOrCreate()

In [0]:
df = spark.read.csv('/content/drive/My Drive/Datasets/LoanStats_2018Q4.csv',inferSchema = "True",header = "True")

In [34]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: doubl

In [35]:
df.count()

128416

In [0]:
df_sel = df.select(['term','home_ownership','grade','purpose',
                    'int_rate','addr_state','loan_status','application_type',
                    'loan_amnt','emp_length','annual_inc','dti','delinq_2yrs',
                    'revol_util','total_acc','num_tl_90g_dpd_24m','dti_joint'
                    ])

In [38]:
df_sel.describe().show()

+-------+----------+--------------+------+--------+--------+----------+------------------+----------------+------------------+----------+------------------+------------------+-------------------+----------+------------------+--------------------+------------------+
|summary|      term|home_ownership| grade| purpose|int_rate|addr_state|       loan_status|application_type|         loan_amnt|emp_length|        annual_inc|               dti|        delinq_2yrs|revol_util|         total_acc|  num_tl_90g_dpd_24m|         dti_joint|
+-------+----------+--------------+------+--------+--------+----------+------------------+----------------+------------------+----------+------------------+------------------+-------------------+----------+------------------+--------------------+------------------+
|  count|    128412|        128412|128412|  128412|  128412|    128412|            128412|          128412|            128412|    128412|            128412|            128175|             128412|    128

In [43]:
df_sel.describe(['term','loan_amnt','emp_length','annual_inc','dti','delinq_2yrs','revol_util','total_acc']).show()

+-------+----------+------------------+----------+------------------+------------------+-------------------+----------+------------------+
|summary|      term|         loan_amnt|emp_length|        annual_inc|               dti|        delinq_2yrs|revol_util|         total_acc|
+-------+----------+------------------+----------+------------------+------------------+-------------------+----------+------------------+
|  count|    128412|            128412|    128412|            128412|            128175|             128412|    128256|            128412|
|   mean|      null| 15971.32102139987|      null|  82797.3278609476| 19.93317753071978|0.22783696227766875|      null|22.677413325857398|
| stddev|      null|10150.384232741852|      null|108298.46579150103|20.143542243475455| 0.7337929617806055|      null|12.129215673024776|
|    min| 36 months|              1000|    1 year|               0.0|               0.0|                  0|        0%|                 2|
|    max| 60 months|       

In [44]:
df_sel.cache                                                                    #cache the dataset in executer cache memory.

<bound method DataFrame.cache of DataFrame[term: string, home_ownership: string, grade: string, purpose: string, int_rate: string, addr_state: string, loan_status: string, application_type: string, loan_amnt: int, emp_length: string, annual_inc: double, dti: double, delinq_2yrs: int, revol_util: string, total_acc: int, num_tl_90g_dpd_24m: int, dti_joint: double]>

In [45]:
df_sel.describe(['loan_amnt','emp_length','dti','delinq_2yrs','revol_util','total_acc']).show()

+-------+------------------+----------+------------------+-------------------+----------+------------------+
|summary|         loan_amnt|emp_length|               dti|        delinq_2yrs|revol_util|         total_acc|
+-------+------------------+----------+------------------+-------------------+----------+------------------+
|  count|            128412|    128412|            128175|             128412|    128256|            128412|
|   mean| 15971.32102139987|      null| 19.93317753071978|0.22783696227766875|      null|22.677413325857398|
| stddev|10150.384232741852|      null|20.143542243475455| 0.7337929617806055|      null|12.129215673024776|
|    min|              1000|    1 year|               0.0|                  0|        0%|                 2|
|    max|             40000|       n/a|             999.0|                 24|    99.90%|               160|
+-------+------------------+----------+------------------+-------------------+----------+------------------+



In [51]:
df.createOrReplaceTempView('loanStats')         #Temp table
spark.sql('select emp_length from loanstats limit 10').show()

+----------+
|emp_length|
+----------+
|  < 1 year|
|       n/a|
|   3 years|
| 10+ years|
| 10+ years|
|   6 years|
|   5 years|
| 10+ years|
| 10+ years|
|  < 1 year|
+----------+



In [0]:
from pyspark.sql.functions import regexp_extract,regexp_replace
from pyspark.sql.functions import col